<a href="https://colab.research.google.com/github/shaoni11/Coursera_Capstone/blob/master/bank_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pandas_profiling --upgrade
#!pip install klib
#!pip install sweetviz
import sys
#!{sys.executable} -m pip install -U pandas-profiling[notebook]
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import pandas as pd
import numpy as np
import klib
import pandas_profiling
from pandas_profiling import ProfileReport
import sweetviz
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from platform import python_version
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import *
from sklearn.metrics import *

from sklearn import set_config 
from sklearn.utils import estimator_html_repr 
from IPython.core.display import display, HTML 

set_config(display='diagram')
from platform import python_version

%precision 2
pd.set_option('max_columns',200)
pd.set_option('display.precision',2)
pd.set_option('display.float_format','{:,.2f}'.format)


from sklearn import set_config 
from sklearn.utils import estimator_html_repr 
from IPython.core.display import display, HTML 

set_config(display='diagram')
from platform import python_version

%precision 2

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120
pd.options.display.float_format='{:,.2f}'.format
pd.options.display.precision = 2

np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=5)

sns.set_style("white")

StartBold = "\033[1m"
EndBold = "\033[0m"

print('python',python_version())
print(np.__name__, np.__version__)
print(pd.__name__, pd.__version__)
print(klib.__name__, klib.__version__)
#print(pp.__name__, pp.__version__)
print(sklearn.__name__, sklearn.__version__)
print(sweetviz.__name__, sweetviz.__version__)

pd.set_option('max_columns',200)
pd.set_option('display.precision',2)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/BankChurners.csv', index_col=0).iloc[:, :-2]

# Reset index IDs and drop old index values
df.reset_index(drop=True, inplace=True)

print(df.info())
display(df.sample(5).T)  # Transpose just for easier viewing

In [ ]:
klib.missingval_plot(df)

In [ ]:
df_c = klib.data_cleaning(df, convert_dtypes=False)
print(df_c.info())

In [ ]:
profile = ProfileReport(df, explorative=True)
profile

In [ ]:
profile.to_file("Sales_Analysis.html")
profile.to_file("Sales_Analysis.json")

In [ ]:
# Replace the values in attrition_flag with binary values, 1 for attrited_customer
df_c['attrition_flag'].replace({'Existing Customer':False, 'Attrited Customer':True}, inplace=True)

In [ ]:
df_c.columns

In [ ]:
def plot_silhouettes(data, clusters, metric='euclidean'):
    
    from matplotlib import cm
    from sklearn.metrics import silhouette_samples

    cluster_labels = np.unique(clusters)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = metrics.silhouette_samples(data, clusters, metric='euclidean')
    c_ax_lower, c_ax_upper = 0, 0
    cticks = []
    for i, k in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[clusters == k]
        c_silhouette_vals.sort()
        c_ax_upper += len(c_silhouette_vals)
        color = cm.jet(float(i) / n_clusters)
        plt.barh(range(c_ax_lower, c_ax_upper), c_silhouette_vals, height=1.0, 
                      edgecolor='none', color=color)

        cticks.append((c_ax_lower + c_ax_upper) / 2)
        c_ax_lower += len(c_silhouette_vals)
    
    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg, color="red", linestyle="--") 

    plt.yticks(cticks, cluster_labels)
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')

    plt.tight_layout()
    plt.show()
    
    return

def plot_dendrogram(model, **kwargs):
    from scipy.cluster.hierarchy import dendrogram
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
for i in df_c.columns: # Make a for loops
    if df_c[i].dtype == 'object':
      df_dummies = pd.get_dummies(df_c) 



df_c.card_category = pd.Categorical(
    df_c.card_category,
    ['Blue' , 'Silver' ,'Gold' , 'Platinum'],
    ordered=True)
df_c.income_category = pd.Categorical(
    df_c.income_category,
    ['Unknown' , 'Less than $40K', '$40K - $60K', '$60K - $80K' ,'$80K - $120K' , '$120K +'],
    ordered=True)
df_c.education_level = pd.Categorical(
    df_c.education_level,
    ['Unknown' , 'Uneducated', 'High School', 'College' ,'Graduate' , 'Post-Graduate', 'Doctorate'],
    ordered=True)

df_c.gender = pd.Categorical(df_c.gender)
df_c.marital_status = pd.Categorical(df_c.marital_status)

In [ ]:

# Split data in to Features X and Target y
y = df_c['attrition_flag']
X = df_c.drop(['attrition_flag'], axis=1)
print('X and y shapes:')
print(X.shape,y.shape,'\n')
print('Target Ratio:')
print(y.value_counts(normalize=True, dropna=False),'\n')


In [ ]:
y.value_counts(normalize=True, dropna=False).plot.bar(title='attrited_customer distribution');

In [ ]:
y.value_counts(normalize=True, dropna=False)

In [ ]:
X.describe().T

In [ ]:
features_numeric     = list(df_c[["customer_age","dependent_count","months_on_book","total_relationship_count","months_inactive_12_mon",
           "contacts_count_12_mon","credit_limit","total_revolving_bal","avg_open_to_buy","total_amt_chng_q4_q1",
           "total_trans_amt","total_trans_ct","total_ct_chng_q4_q1","avg_utilization_ratio"]])
features_nominal     = ['gender','marital_status']
features_nominal_ohe = ['is_Female', 'is_Male', 'is_Divorced', 'is_Married', 'is_Single','is_Unknown_Marital'] # OneHotEncoded
features_ordinal     = ['education_level','income_category','card_category']
features_names       = df_c.columns[:19].values
features_names_enc   =  features_numeric + features_ordinal + features_nominal_ohe

In [ ]:
#Scale & Transform

numeric_transformer = StandardScaler()
ordinal_transformer = OrdinalEncoder()
nominal_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, features_numeric),
        ('ord', ordinal_transformer, features_ordinal),
        ('nom', nominal_transformer, features_nominal),
    ],
    remainder='passthrough',
    n_jobs=-1
)

X_ = preprocessor.fit_transform(X)
X_enc = pd.DataFrame(X_, columns=features_names_enc, index=X.index)

In [ ]:
corr = X_enc.join(y).corr() # Join with y to be included in correlations

fig, ax = plt.subplots(figsize=(12, 12))
mask = np.triu(np.ones_like(corr, dtype=bool))
hm = sns.heatmap(corr, annot=True, center=0, linecolor='lightgray', linewidths=.01, ax=ax,
                 cmap="coolwarm", fmt='.2g', square=True, cbar=False,
                 mask=((abs(corr)<=.6) | mask))
plt.title('Correlation Heatmap (r>abs(.6))', fontsize=24);

In [ ]:
#Feature Analysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier())
    ])

clf.fit(X, y)

print("model score: %.3f" % clf.score(X, y))
print(classification_report(y,clf.predict(X)))

In [ ]:

from sklearn.tree import export_graphviz, plot_tree

s = export_graphviz(clf.named_steps.classifier,            
                feature_names = features_names_enc, 
                class_names= ['attrited','existing'],
                filled = True,
                max_depth = 3
               )
graphviz.Source(s)

In [ ]:
pd.DataFrame(clf.named_steps.classifier.feature_importances_,
             index=features_names_enc,
             columns=['Importance']
            ).sort_values('Importance',ascending=False)

In [ ]:
#Cluster Analysis

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA,KernelPCA, TruncatedSVD

from sklearn import metrics

In [ ]:
#Trying K-Means
pl_kmeans = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('cluster', KMeans(n_clusters=2))
    ])

clusters = pl_kmeans.fit_predict(X)
kmeans = pl_kmeans.named_steps.cluster

print('completeness_score: {:.2f}'.format(completeness_score(y,clusters)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clusters)))

In [ ]:
plot_silhouettes(pl_kmeans.transform(X),clusters)

In [ ]:
# Check the clusters assignment distributions
clusters_df = pd.DataFrame(clusters, columns=["Cluster"])['Cluster']
clusters_df.value_counts()

In [ ]:
#Cluster assignment representation:

sns.pairplot(
    x_vars=['months_on_book','avg_open_to_buy','avg_utilization_ratio','total_trans_ct'],
    y_vars=['customer_age','credit_limit','total_revolving_bal','total_trans_amt'],
    hue='Cluster',
    height=3,
    data=X_enc.reset_index(drop=True).join(clusters_df)
    );

In [ ]:
# Elbow-curve/SSD
ssd = []
range_n_clusters = range(2,15)
for num_clusters in range_n_clusters:
    pl_kmeans = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('cluster', KMeans(n_clusters=num_clusters))
    ])
    
    pl_kmeans.fit(X)
    kmeans = pl_kmeans.named_steps.cluster
    ssd.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters
plt.plot(ssd);

In [ ]:
# Silhouette Analysis
range_n_clusters = range(2,15)
for num_clusters in range_n_clusters:
    
    # Initialise kmeans
    pl_kmeans = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('cluster', KMeans(n_clusters=num_clusters))
    ])
    
    pl_kmeans.fit(X)
    kmeans = pl_kmeans.named_steps.cluster

    cluster_labels = kmeans.labels_
    
    # Silhouette Score
    silhouette_avg = silhouette_score(pl_kmeans.transform(X), cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
#Trying agglomerative Clustering
pl_kmeans = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('cluster', AgglomerativeClustering(n_clusters=2))
    ])

clusters = pl_kmeans.fit_predict(X)
kmeans = pl_kmeans.named_steps.cluster

print('completeness_score: {:.2f}'.format(completeness_score(y,clusters)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clusters)))

In [ ]:
plot_silhouettes(X_enc,clusters)

In [ ]:

# Check the clusters assignment distributions
clusters_df = pd.DataFrame(clusters, columns=["Cluster"])['Cluster']
clusters_df.value_counts()

In [ ]:
sns.lmplot(x="total_trans_ct", y="total_trans_amt", hue="Cluster",
           data=X_enc.reset_index(drop=True).join(clusters_df));

In [ ]:
sns.pairplot(
    x_vars=['months_on_book','avg_open_to_buy','avg_utilization_ratio','total_trans_ct'],
    y_vars=['customer_age','credit_limit','total_revolving_bal','total_trans_amt'],
    hue='Cluster',
    height=3,
    data=X_enc.reset_index(drop=True).join(clusters_df)
    );

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model = model.fit(X_enc)

plt.subplots(figsize=(20,5))
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=1)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
#Applying KernelPCA

_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.scatter(xs, ys, zs, marker=m)
ax.scatter(_.iloc[:,0],_.iloc[:,1],_.iloc[:,2],c=y, alpha=.6);

In [ ]:
_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = PCA(3).fit_transform(_)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.scatter(xs, ys, zs, marker=m)
ax.scatter(_[:,0],_[:,1],_[:,2],c=y, alpha=.6);

In [ ]:
_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(3, kernel='poly', degree=5).fit_transform(_)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(_[:,0],_[:,1],_[:,2],c=y, alpha=.6);

In [ ]:

_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(3, kernel='sigmoid').fit_transform(_)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(_[:,0],_[:,1],_[:,2],c=y, alpha=.6);

In [ ]:

_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(3, kernel='sigmoid').fit_transform(_)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(_[:,0],_[:,1],_[:,2],c=y, alpha=.6);

In [ ]:

_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(3, kernel='rbf', gamma=5).fit_transform(_)
clst = KMeans(2).fit_predict(_)
print('completeness_score: {:.2f}'.format(completeness_score(y,clst)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clst)))
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
plt.scatter(_[:,0],_[:,1],c=y, alpha=.6);

In [ ]:
plt.scatter(_[:,0],_[:,1],c=clst, alpha=.6);

In [ ]:
for i in np.arange(1,10,1):
    _ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
    _ = KernelPCA(3, kernel='rbf', gamma=i).fit_transform(_)
    clst = KMeans(2).fit_predict(_)
    print(i)
    print('completeness_score: {:.2f}'.format(completeness_score(y,clst)))
    print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clst)))
    print()

In [ ]:
_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(3, kernel='poly', degree=5).fit_transform(_)
clst = AgglomerativeClustering().fit_predict(_)
print('completeness_score: {:.2f}'.format(completeness_score(y,clst)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clst)))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(_[:,0],_[:,1],_[:,2],c=clst, alpha=.6);

In [ ]:
_ = df_c[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(2, kernel='poly').fit_transform(_)
clst = AgglomerativeClustering(2).fit_predict(_)
print('completeness_score: {:.2f}'.format(completeness_score(y,clst)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clst)))
plt.scatter(_[:,0],_[:,1],c=y, alpha=.6);

In [ ]:
plt.scatter(_[:,0],_[:,1],c=clst, alpha=.6);

In [ ]:
_ = X_enc[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(2, kernel='poly').fit_transform(_)
clst = DBSCAN(metric='cosine').fit_predict(_)
print('completeness_score: {:.2f}'.format(completeness_score(y,clst)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clst)))
plt.scatter(_[:,0],_[:,1],c=clst, alpha=.6);

In [ ]:
_ = df_c[['total_trans_amt','total_trans_ct','total_revolving_bal']]
_ = KernelPCA(3, kernel='poly').fit_transform(_)
clst = AgglomerativeClustering(2).fit_predict(_)
print('completeness_score: {:.2f}'.format(completeness_score(y,clst)))
print('homogeneity_score:  {:.2f}'.format(homogeneity_score(y,clst)))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.scatter(xs, ys, zs, marker=m)
ax.scatter(_[:,0],_[:,1],_[:,2],c=y, alpha=.6);

In [ ]:
# Global Parameters

random_state = 2021
n_jobs = 4

In [ ]:
#Helper Function
def get_GS_cv_results(gs):
    '''
        Extract Grid Seach Results
        Author: Mourad Askar
    '''
    _cv_results_ = gs.cv_results_
    _scorer_keys = gs.scorer_.keys()

    df_gs_scores = pd.DataFrame()

    for k in _cv_results_.keys():
        if k.startswith('param_'):
            param_key = ('param',k.split('_',1)[1])
            param_values = _cv_results_[k]
            df_gs_scores = pd.concat([df_gs_scores,pd.DataFrame({param_key:param_values})], axis=1)

        elif k.startswith('mean_train') or k.startswith('mean_test'):
            score_key = (k.split('_',2)[2],k.split('_',2)[1])
            score_results = _cv_results_[k]
            df_gs_scores = pd.concat([df_gs_scores,pd.DataFrame({score_key:score_results})], axis=1)

    return df_gs_scores

def display_grid_search_scores_grid(gs):
    '''
        Display Grid Search Scores Table
        Author: Mourad Askar
    '''
    _cv_results_ = gs.cv_results_
    _scorer_keys = gs.scorer_.keys()
    _param_keys = list(_cv_results_['params'][0].keys())
    df_gs_scores = get_GS_cv_results(gs)
    
    display(df_gs_scores)
    

def plot_grid_search_results(gs, plot_scoring='f1'):
    '''
        Plot Grid Search Train/Val Scores
        Author: Mourad Askar
    '''
    _cv_results_ = gs.cv_results_
    _scorer_keys = gs.scorer_.keys()

    if(type(gs) == GridSearchCV):
        model = gs.best_estimator_
        if(type(model) == Pipeline):
            model = gs.best_estimator_.named_steps.classifier
    else:
        model = gs
    
    print(f'{StartBold}Estimator:{EndBold}')
    display(model)
    print()
    print(f'{StartBold}Best Result (Suggested):{EndBold}')
    print(f'{StartBold}\t{"":20} {"train":>6} {"valdn":>6}{EndBold}')
    for _score in _scorer_keys:
        print('\t{2:>20} {0:>6.2f} {1:>6.2f}'.format(_cv_results_['mean_train_' + _score][gs.best_index_],
                                             _cv_results_['mean_test_' + _score][gs.best_index_],
                                             _score))
    print()
    print(f'{StartBold}Params:{EndBold}')
    print('\t{}'.format(gs.best_params_))
    print()
    
 
    scoring_label = plot_scoring
    not_negative = 1
    if plot_scoring.startswith('neg_'):
        scoring_label = plot_scoring[4:]
        not_negative = -1
    df = pd.DataFrame({
        'params': _cv_results_['params'],
        'mean_train_' + plot_scoring: _cv_results_['mean_train_' + plot_scoring] * not_negative,
        'mean_test_' + plot_scoring: _cv_results_['mean_test_' + plot_scoring] * not_negative
    })
    fig, ax = plt.subplots()
    df.plot('params',['mean_train_' + plot_scoring,'mean_test_' + plot_scoring], 'line', ax=ax)
    ax.axvline(gs.best_index_,0,1, color='r', linestyle='--')
    plt.grid(axis='both',c='lightgrey',ls=':')
    plt.xticks(ticks=range(df.params.count()), labels=df.index)
    plt.tick_params(axis='x', rotation=0)
    plt.xlabel('params index')
    plt.ylabel(scoring_label)
    plt.legend(['train','valdn'])
    plt.show()
    
    #_ = gs
    #df_ = pd.DataFrame(_.cv_results_).filter(regex='(?:mean_test)|(?:params)')
    #df_.insert(0,'classifier',_.best_estimator_.named_steps.classifier.__class__.__name__)
    #d_result = {_.best_estimator_.named_steps.classifier:dict(df_.iloc[_.best_index_])}
    #d_result
    display_grid_search_scores_grid(gs)

    if("beep" in globals()): beep(True)
        
def score_classification_model(fitted_model, X, y_true, return_score=False):
    '''
        Print Classification Metrics
        Author: Mourad Askar
    '''
    p = fitted_model.predict(X)
    
    accuracy = metrics.accuracy_score(y_true, p)
    f1 = metrics.f1_score(y_true, p)
    recall = metrics.recall_score(y_true, p)
    precision = metrics.precision_score(y_true, p)
    balanced_accuracy = metrics.balanced_accuracy_score(y_true, p)
    roc = metrics.roc_auc_score(y_true, p)

    print('\taccuracy score:          {:>5.2f}'.format(accuracy))
    print('\tf1 score:                {:>5.2f}'.format(f1))
    print('\trecall score:            {:>5.2f}'.format(recall))
    print('\tprecision score:         {:>5.2f}'.format(precision))
    print('\tbalanced_accuracy score: {:>5.2f}'.format(balanced_accuracy))
    print('\troc_auc score:           {:>5.2f}'.format(roc))

    #print(metrics.classification_report(y_true,p))
    metrics.plot_confusion_matrix(fitted_model,X, y_true)
    plt.show()
          
    #print(metrics.classification_report(y_true,p, sample_weight=compute_sample_weight('balanced',y_true)))
    #metrics.plot_confusion_matrix(fitted_model,X, y_true, sample_weight=compute_sample_weight('balanced',y_true))

    if((type(return_score) is str) or return_score):
        if(type(fitted_model) == GridSearchCV):
            model = fitted_model.best_estimator_
            if(type(model) == Pipeline):
                model = fitted_model.best_estimator_.named_steps.classifier
        else:
            model = fitted_model
        key = str(model).replace('\n','') + ( (' - ' + return_score) if type(return_score) == str else '')
        results_dict = {key : {}}
        results_dict[key]['Estimator'] = model.__class__.__name__
        results_dict[key]['TAG'] = return_score if type(return_score) == str else '-'
        results_dict[key]['Accuracy'] = accuracy
        results_dict[key]['F1'] = f1
        results_dict[key]['Recall'] = recall
        results_dict[key]['Precision'] = precision
        results_dict[key]['Balanced_Accuracy'] = balanced_accuracy
        results_dict[key]['ROC_AUC'] = roc
        return(results_dict)
    
def plot_coefficients(coef, feature_names, top_n=0):
    '''
        Plot Coefficents Magnitude
        Author: Mourad Askar
    '''
    _ = pd.DataFrame({'features':feature_names,'coef':coef}).sort_values('coef',key=lambda x: abs(x),ascending=False)
    if top_n > 0:
        _ = _.iloc[:top_n,:]
    n_features = _.shape[0]
    #plt.subplots(figsize=(10,20))
    plt.barh(range(n_features), _['coef'], align='center')
    plt.yticks(np.arange(n_features), _['features'])
    plt.xlabel('Value')
    plt.ylabel('Feature')
    plt.ylim(n_features,-1)
    plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC

In [ ]:
numeric_transformer = StandardScaler()
nominal_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, features_numeric),
        ('nominal', nominal_transformer, features_nominal),
        ('ordinal', ordinal_transformer, features_ordinal),
    ])

clf = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', SGDClassifier())
    ])

display(clf)

In [ ]:
# Common Grid Search Parameters that will be use across all models

grid_search_defaults = {
    'cv': 5,
    'scoring': ['f1', 'recall', 'precision', 'balanced_accuracy', 'accuracy', 'roc_auc'],
    'refit': 'f1',
    'return_train_score': True,
    'error_score': 0,
    'verbose': 3,
    'n_jobs': n_jobs,
}

In [ ]:
#Modeling
# Dictionary to collect the scores from different models
scores_TestSet = {}

In [ ]:
#SGD Classifier
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=101,test_size=0.3)
cl_pl = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', SGDClassifier(loss='log',n_jobs=n_jobs,class_weight={1:2},random_state=random_state))
    ]
)

params = {
#    'classifier__class_weight': ['balanced',{1:2},{1:3},{1:4}],
    'classifier__l1_ratio': [0,.25,.5,.75,1],
    'classifier__alpha': np.logspace(-4,-1,5).round(5),
#    'classifier__alpha': [.0006,.0007,.0008],
}
gs = GridSearchCV(cl_pl, params, **grid_search_defaults)
gs.fit(X,y)

print(f'\n\n{StartBold}Train_Set Tuning Results:{EndBold}')
plot_grid_search_results(gs, grid_search_defaults['refit'])
print(f'\n\n{StartBold}Test_Set Results:{EndBold}')
results = score_classification_model(gs, X_test, y_test, True)
scores_TestSet.update(results)

In [ ]:
# Decision Tree Classifier
cl_pl = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(class_weight={1:2}, random_state=random_state))
    ]
)

params = {
#    'classifier__class_weight': ['balanced',{1:2},{1:3},{1:4}],
    'classifier__max_depth': [6,7,8],
    'classifier__min_samples_split': [0,.01,.02,.03],
#    'classifier__max_features': [.1,.5,1.0],
}
gs = GridSearchCV(cl_pl,params, **grid_search_defaults)
gs.fit(X,y)

print(f'\n\n{StartBold}Train_Set Tuning Results:{EndBold}')
plot_grid_search_results(gs, grid_search_defaults['refit'])

print(f'\n\n{StartBold}Test_Set Results:{EndBold}')
results = score_classification_model(gs, X_test, y_test, True)
scores_TestSet.update(results)

In [ ]:
#Randoem Forest Classifier

cl_pl = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(class_weight={1:2},n_jobs=-1,random_state=random_state))
    ]
)

params = {
#    'classifier__class_weight': ['balanced','balanced_subsample',{1:2},{1:3},{1:4}],
#    'classifier__n_estimators': [200,300,400,500],
    'classifier__max_depth': [24], #[16,24,32],
    'classifier__min_samples_split': [.01,.05,.1],
    'classifier__max_features': [.1,.5,1.0],
}
gs = GridSearchCV(cl_pl,params, **grid_search_defaults)
gs.fit(X,y)

print(f'\n\n{StartBold}Train_Set Tuning Results:{EndBold}')
plot_grid_search_results(gs, grid_search_defaults['refit'])

print(f'\n\n{StartBold}Test_Set Results:{EndBold}')
results = score_classification_model(gs, X_test, y_test, True)
scores_TestSet.update(results)

In [ ]:
pd.DataFrame(gs.best_estimator_.named_steps.classifier.feature_importances_,
             index=features_names_enc,
             columns=['Importance']
            ).sort_values('Importance',ascending=False)

In [ ]:

pd.DataFrame(scores_TestSet).T.drop(columns='TAG')